In [1]:
from tensorflow import keras
import dlib

model = keras.models.load_model('model.h5')
detector = dlib.get_frontal_face_detector()

expression_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}
colors = [(0,0,255),(0,0,0),(135,138,128),(18,153,255),(255,0,0),(240,32,160),(0,255,0)]

In [2]:
def detect_faces(frame):
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face = detector(frame)
    faces = []

    for i in range(len(face)):
        x = face[i].left()
        y = face[i].top()
        w = face[i].right()-face[i].left()
        h = face[i].bottom()-face[i].top()
        if x<0:
            x = 0
        if y<0:
            y = 0
        faces.append((x, y, w, h))
        
    return gray, faces

In [3]:
def face_post(frame, bottom_right, width, prediction):
    
    if prediction =="angry":
        img = cv2.imread("Image_fer/00_Angry_Second.png")
    if prediction =="disgusted":
        img = cv2.imread("Image_fer/01_Disgust_Second.png")
    if prediction =="fearful":
        img = cv2.imread("Image_fer/02_Fear_Second.png")
    if prediction =="happy":
        img = cv2.imread("Image_fer/03_Happy_Second.png")
    if prediction =="sad":
        img = cv2.imread("Image_fer/04_Sad_Second.png")
    if prediction =="surprised":
        img = cv2.imread("Image_fer/05_Surprise_Second.png")
    if prediction =="neutral":
        img = cv2.imread("Image_fer/06_Neutral_Second.png")
    
    img_wh = int(width/2)
    img = cv2.resize(img, (img_wh,img_wh))
        
    img_top_left = (int(bottom_right[0]),int(bottom_right[1]-img_wh))
    img_bottom_right = (int(bottom_right[0]+img_wh),int(bottom_right[1]))

    if img_top_left[0]+img_wh<frame.shape[1] and img_top_left[1]+img_wh<frame.shape[0]:
    
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(img_gray, 25, 255, cv2.THRESH_BINARY_INV)
        area = frame[img_top_left[1]: img_top_left[1]+img_wh, img_top_left[0]: img_top_left[0]+img_wh]
        area_no = cv2.bitwise_and(area,area,mask=mask)
        final = cv2.add(area_no, img)
        frame[img_top_left[1]: img_top_left[1]+img_wh, img_top_left[0]: img_top_left[0]+img_wh] = final

    return frame

In [4]:
def face_check(frame, model):
    
    gray, faces = detect_faces(frame)
    data = {"faces" : []}
    
    for i, (x, y, w, h) in enumerate(faces):
        
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)

        prediction = model.predict(cropped_img)[0]
        pred_array = prediction.tolist()
        
        predictions = {
            "angry": float(prediction[0]),
            "disgusted": float(prediction[1]),
            "fearful": float(prediction[2]),
            "happy": float(prediction[3]),
            "sad": float(prediction[4]),
            "surprised": float(prediction[5]),
            "neutral": float(prediction[6])
        }
        
        data["faces"].append({"id" : i,"xywh": (x, y, w, h),"predictions": predictions})
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 4)
        cv2.putText(frame,"{}: {}".format(max(predictions, key=predictions.get),
                                          round(predictions[max(predictions, key=predictions.get)],3)),
                    (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,colors[pred_array.index(max(pred_array))],2)
        
        frame = face_post(frame, (x+w, y+h), w, max(predictions, key=predictions.get))
        
    return data, frame

In [5]:
import cv2
import numpy as np

VIDEO_IN = cv2.VideoCapture(0)

while True:
    hasFrame, img = VIDEO_IN.read()
    img = cv2.resize(img, None, fx=0.8, fy=0.8)
    data, img = face_check(img, model)
    cv2.imshow("Frame", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
VIDEO_IN.release()
cv2.destroyAllWindows()